In [15]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import pandas as pd
from zipfile import ZipFile
from sklearn.model_selection import train_test_split
from pns_dict import PNS_DICT_RAW
import numpy as np

PNS_DATA = Path('../data/raw/PNS_2019/PNS_2019.zip').resolve()
zip_pense = ZipFile(PNS_DATA)

path_unzip = zip_pense.namelist()[0]
df = pd.read_csv(zip_pense.open(path_unzip), delimiter=';', low_memory=False)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
print("O dataset PNS possui {} linhas e {} colunas (features).".format(*df.shape))

O dataset PNS possui 293726 linhas e 1087 colunas (features).


Neste projeto usaremos o PHQ9 https://www.mdcalc.com/calc/1725/phq9-patient-health-questionnaire9 para ter um indicador se a possa esta depressiva ou não no momento, além deste indicador temos também a questão Q092 (Algum médico ou profissional de saúde mental (como psiquiatra ou psicólogo) já lhe deu o diagnóstico de depressão?) como referência de filtragem de dados, questões mais específicas como 'Por causa da depressão Toma medicamentos', que também poderiam ser utilizadas como parâmetro de indicativo de depressão são, na verdade, dependentes da variável Q092.

### Criando o resultado PHQ9

In [17]:
df.loc[(18 <= df['C008']) & (df['C008'] <= 59), 'phq9_total'] = df.loc[(18 <= df['C008']) & (df['C008'] <= 59), ['N010', 'N011', 'N012', 'N013', 'N014', 'N015', 'N016', 'N017', 'N018']].sum(axis=1, min_count=1) - 9 # -9 correção do valor, como o questionario a resposta "Nenhuma" começa no 1

# Classificando a gravidade da depressão com base na pontuação total
def classify_depression(row):
    value = row['phq9_total']

    if  0 <= value < 5:
        return 1
    elif value < 10:
        return 2
    elif value < 15:
        return 3
    elif value < 20:
        return 4
    elif value >= 20:
        return 5

    return np.nan # questionario não respondido ou idade não compativel

df['depression_severity'] = df.apply(classify_depression, axis=1)

In [18]:
selected = df[df['depression_severity'].notna()] # pega somente os registros que foram respondidos como sim ou não
print("A quantidade de dados investigável dos dados são {}, {:02f}% do total.".format(selected.shape[0], 100*selected.shape[0]/df.shape[0]))

A quantidade de dados investigável dos dados são 65803, 22.402852% do total.


In [19]:
selected = df[(df['depression_severity'].notna()) | df['Q092'].isin([1, 2])] # pega somente os registros que foram respondidos como sim ou não ou que possui informações sobre dignostico de depressao
print("A quantidade de dados investigável dos dados são {}, {:02f}% do total.".format(selected.shape[0], 100*selected.shape[0]/df.shape[0]))

A quantidade de dados investigável dos dados são 90846, 30.928825% do total.


In [20]:
print("Numero de casas entrevistadas: ", selected.groupby(['V0001', 'V0024', 'UPA_PNS', 'V0006_PNS']).size().reset_index().shape[0])
print("Numero de pessoas entrevistadas: ", selected.groupby(['V0001', 'V0024', 'UPA_PNS', 'V0006_PNS', 'C00301']).size().reset_index().shape[0])

Numero de casas entrevistadas:  90846
Numero de pessoas entrevistadas:  90846


#### Filtragem e renomeação das colunas de interesse

In [21]:
selected_columns = {
    # Parte 1 - Identificação e Controle
    'V0001': 'uf',
    'V0022': 'moradores',
    'V0026': 'tipo_zona',

    # VARIÁVEIS DERIVADAS
    'VDF004': 'renda_pc',

    # Módulo A - Informações do Domicílio
    'A001': 'casa_tipo',
    'A02201': 'casa_animal',

    # Módulo B - Visitas domiciliares de Equipe de Saúde da Família e Agentes de Endemias
    'B001': 'cad_sus',

    # Módulo C - Características gerais dos moradores
    'C006': 'sexo',
    'C008': 'idade',
    'C009': 'cor',
    'C01001': 'vive_conjugue',

    # Módulo D - Características de educação dos moradores
    'D001': 'sabe_ler',
    'D00901': 'escolaridade',

    # Módulo P - Estilos de vida
    'P00104': 'peso',
    'P00404': 'altura',

    'P006': 'freq_feijao_sem',
    'P00901': 'freq_verde_sem',
    'P01101': 'freq_carne_vermelha_sem',
    'P013': 'freq_frango_sem',
    'P015': 'freq_peixe_sem',
    'P018': 'freq_fruta_sem',
    'P019': 'freq_fruta_dia',
    'P023': 'freq_leite_sem',


    'Q092': 'teve_depre',

    'phq9_total': 'phq9_total',
    'depression_severity': 'depression_severity',

}
{k: PNS_DICT_RAW.get_question(k) for k in selected_columns.keys()}

{'V0001': 'Unidade da Federação',
 'V0022': 'Total de moradores',
 'V0026': 'Tipo de situação censitária',
 'VDF004': 'Faixa de rendimento domiciliar per capita (exclusive o rendimento das pessoas cuja condição na unidade domiciliar era pensionista, empregado doméstico ou parente do empregado doméstico)',
 'A001': 'Tipo do domicílio',
 'A02201': 'Em seu domicílio, há algum animal de estimação',
 'B001': 'O seu domicílio está cadastrado na unidade de saúde da família',
 'C006': 'Sexo',
 'C008': 'Idade do morador na data de referência',
 'C009': 'Cor ou raça',
 'C01001': 'Cônjuge ou companheiro(a) mora em nesse domicílio.',
 'D001': 'Sabe ler e escrever',
 'D00901': 'Qual foi o curso mais elevado que ___frequentou',
 'P00104': 'Peso - Final (em kg)\n(3 inteiros e 1 casa decimal)',
 'P00404': 'Altura - Final (em cm)\n(3 inteiros)',
 'P006': 'Em quantos dias da semana o(a) Sr(a) costuma comer feijão?',
 'P00901': 'Em quantos dias da semana, o(a) Sr(a) costuma comer pelo menos um tipo de ve

In [22]:
df_all = selected[selected_columns.keys()]
df_all = df_all.rename(columns=selected_columns)
df_all

,uf,moradores,tipo_zona,renda_pc,casa_tipo,casa_animal,cad_sus,sexo,idade,cor,...,freq_verde_sem,freq_carne_vermelha_sem,freq_frango_sem,freq_peixe_sem,freq_fruta_sem,freq_fruta_dia,freq_leite_sem,teve_depre,phq9_total,depression_severity
0,11,6.0,1,2.0,1.0,1.0,1.0,2.0,55.0,1.0,...,1.0,3.0,6.0,3.0,3.0,NaN,7.0,1.0,6.0,2.0
9,11,4.0,1,2.0,1.0,1.0,1.0,2.0,19.0,4.0,...,4.0,4.0,2.0,0.0,0.0,NaN,0.0,2.0,2.0,1.0
10,11,8.0,1,3.0,1.0,1.0,1.0,2.0,45.0,2.0,...,7.0,7.0,0.0,3.0,7.0,2.0,0.0,2.0,3.0,1.0
18,11,1.0,1,2.0,1.0,1.0,1.0,2.0,58.0,2.0,...,3.0,7.0,3.0,3.0,3.0,NaN,0.0,2.0,0.0,1.0
19,11,2.0,1,1.0,2.0,2.0,1.0,2.0,28.0,4.0,...,3.0,3.0,2.0,1.0,3.0,NaN,7.0,2.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293709,53,3.0,2,4.0,1.0,1.0,2.0,2.0,54.0,4.0,...,7.0,3.0,2.0,0.0,2.0,NaN,0.0,2.0,4.0,1.0
293712,53,2.0,2,4.0,1.0,1.0,3.0,1.0,44.0,4.0,...,7.0,7.0,7.0,2.0,3.0,NaN,0.0,2.0,1.0,1.0
293713,53,4.0,2,2.0,1.0,1.0,1.0,2.0,32.0,4.0,...,2.0,6.0,0.0,0.0,1.0,NaN,7.0,2.0,1.0,1.0
293718,53,3.0,2,5.0,1.0,1.0,2.0,1.0,54.0,4.0,...,6.0,3.0,3.0,0.0,3.0,NaN,0.0,1.0,4.0,1.0


In [23]:
missing_values = df_all.isna().sum()
missing_values

uf                             0
moradores                      0
tipo_zona                      0
renda_pc                      22
casa_tipo                      0
casa_animal                    0
cad_sus                        0
sexo                           0
idade                          0
cor                            0
vive_conjugue                  0
sabe_ler                       0
escolaridade               13965
peso                         892
altura                       892
freq_feijao_sem                0
freq_verde_sem                 0
freq_carne_vermelha_sem        0
freq_frango_sem                0
freq_peixe_sem                 0
freq_fruta_sem                 0
freq_fruta_dia             50441
freq_leite_sem                 0
teve_depre                     0
phq9_total                 25043
depression_severity        25043
dtype: int64

In [24]:
df_all[df_all['escolaridade'].isna()]

,uf,moradores,tipo_zona,renda_pc,casa_tipo,casa_animal,cad_sus,sexo,idade,cor,...,freq_verde_sem,freq_carne_vermelha_sem,freq_frango_sem,freq_peixe_sem,freq_fruta_sem,freq_fruta_dia,freq_leite_sem,teve_depre,phq9_total,depression_severity
19,11,2.0,1,1.0,2.0,2.0,1.0,2.0,28.0,4.0,...,3.0,3.0,2.0,1.0,3.0,NaN,7.0,2.0,0.0,1.0
44,11,4.0,1,7.0,1.0,1.0,2.0,1.0,35.0,4.0,...,5.0,7.0,3.0,1.0,2.0,NaN,0.0,2.0,6.0,2.0
57,11,1.0,1,4.0,2.0,2.0,2.0,2.0,32.0,1.0,...,7.0,7.0,0.0,0.0,3.0,NaN,7.0,2.0,0.0,1.0
65,11,3.0,1,5.0,2.0,2.0,2.0,2.0,20.0,4.0,...,7.0,2.0,2.0,1.0,4.0,NaN,7.0,2.0,0.0,1.0
76,11,3.0,1,3.0,1.0,1.0,3.0,1.0,40.0,4.0,...,3.0,3.0,3.0,2.0,4.0,NaN,5.0,2.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293565,53,4.0,2,2.0,1.0,1.0,1.0,2.0,22.0,4.0,...,7.0,4.0,3.0,0.0,7.0,1.0,7.0,2.0,1.0,1.0
293572,53,1.0,2,4.0,1.0,1.0,1.0,1.0,70.0,4.0,...,7.0,0.0,3.0,0.0,2.0,NaN,7.0,2.0,NaN,NaN
293589,53,3.0,2,3.0,1.0,1.0,3.0,1.0,22.0,2.0,...,4.0,3.0,2.0,2.0,3.0,NaN,2.0,2.0,0.0,1.0
293614,53,4.0,2,7.0,1.0,1.0,3.0,2.0,45.0,1.0,...,5.0,3.0,3.0,1.0,7.0,1.0,7.0,2.0,0.0,1.0


In [25]:
n_nan_cols = (df_all.isna().mean() <= 0.2).sum()
print(n_nan_cols)

23


#### Criando dicionarios

In [26]:
desc_dict = {v: {'origin': k} for k, v in selected_columns.items()}

desc_dict.update({
    'phq9_total': {
        'origin': '*',
        'desc': "Valor de risco para depressão segundo questionario PHQ9",
    },
    'depression_severity': {
        'origin': '*',
        'desc': "Classificação risco de depressão",
        'pv': {1: 'Nenhum ou mínimo', 2: 'Leve', 3: 'Moderada', 4: 'Moderadamente Grave', 5: 'Grave'}
    }
})

df_dict = pd.DataFrame.from_dict(PNS_DICT_RAW.complete_dictionary(desc_dict), orient='index').reset_index()
df_dict.to_csv('../data/processed/PNS_2019/dictionary.csv', index=False)
df_dict

,index,origin,desc,pv
0,uf,V0001,Unidade da Federação,"{11: 'Rondônia', 12: 'Acre', 13: 'Amazonas', 1..."
1,moradores,V0022,Total de moradores,{}
2,tipo_zona,V0026,Tipo de situação censitária,"{1: 'Urbano', 2: 'Rural'}"
3,renda_pc,VDF004,Faixa de rendimento domiciliar per capita (exc...,"{1: 'Até ¼ salário mínimo', 2: 'Mais de ¼ até ..."
4,casa_tipo,A001,Tipo do domicílio,"{1: 'Casa', 2: 'Apartamento', 3: 'Habitação em..."
5,casa_animal,A02201,"Em seu domicílio, há algum animal de estimação","{1: 'Sim', 2: 'Não', 9: 'Ignorado'}"
6,cad_sus,B001,O seu domicílio está cadastrado na unidade de ...,"{1: 'Sim', 2: 'Não', 3: 'Não sabe', 9: 'Ignora..."
7,sexo,C006,Sexo,"{1: 'Homem', 2: 'Mulher'}"
8,idade,C008,Idade do morador na data de referência,{}
9,cor,C009,Cor ou raça,"{1: 'Branca', 2: 'Preta', 3: 'Amarela', 4: 'Pa..."


Vamos embaralhar estes dados e salva-los na pasta processed/PNS/all.zip

In [27]:
df_all = df_all.convert_dtypes()
df_all = df_all.sample(frac=1, random_state=42).reset_index(drop=True)
df_all.to_csv('../data/processed/PNS_2019/all.zip', index=False, compression=dict(method='zip', archive_name='data.csv', compresslevel=9))

Dividindo as base em treino e teste

In [28]:
df_train, df_test = train_test_split(df_all, test_size=0.5, random_state=42)
df_train.to_csv('../data/processed/PNS_2019/train.zip', index=False, compression=dict(method='zip', archive_name='data.csv', compresslevel=9))
df_test.to_csv('../data/processed/PNS_2019/test.zip', index=False, compression=dict(method='zip', archive_name='data.csv', compresslevel=9))

Com isso temos que as pessoas entrevistadas moram em casas diferentes, isso facilita a divisão dos dados.